In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats

In [ ]:
include("/home/chuyu/fish_place_cell_paper/functions/func_map.jl")
include("/home/chuyu/fish_place_cell_paper/functions/func_stat.jl")
include("/home/chuyu/fish_place_cell_paper/functions/func_data.jl")
include("/home/chuyu/fish_place_cell_paper/functions/func_plot.jl")
include("/home/chuyu/fish_place_cell_paper/functions/utils.jl")

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
# rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
experiment_filename_1 = "20230227_155813"
server_1 = 1

experiment_filename_2 = "20230227_165253"
server_2 = 1

experiment_filename_3 = "20230227_175948"
server_3 = 1

experimenter = "chuyu"
analyzer = "chuyu"

In [ ]:
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

ds_save_3 = Dataset(experiment_filename_3, experimenter, gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_3 = Dataset(experiment_filename_3, "chuyu", gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)")
ds_save_analyzer_3 = Dataset(experiment_filename_3, analyzer, gethostname() == "roli-$(server_3)" ? "/data" : "/nfs/data$(server_3)")

In [ ]:

NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all = HDF5.readmmap(NMF_file["Z_all"])
global X_all = HDF5.readmmap(NMF_file["X_all"])
global Y_all = HDF5.readmmap(NMF_file["Y_all"])
global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
close(NMF_file)

n_neuron = length(X_all);

# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)


# for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
whether_tel = falses(n_neuron)
for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
    if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
        whether_tel[which_neuron] = true
    end
end



img_bg_1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;

    
global img_bg_end_1 = img_bg_1[:,:,end]


global w = size(img_bg_end_1, 1)
global l = size(img_bg_end_1, 2)
    

for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_1), "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n100.h5"))
x_bins = read(for_place_calculation_file,"x_bins")
y_bins = read(for_place_calculation_file,"y_bins")
close(for_place_calculation_file)


# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
chamber_roi_1 = read(orientation_correction_file,"chamber_roi")
countour_array_1 = read(orientation_correction_file,"countour_array")
close(orientation_correction_file)


# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_2), "chamber_geometry_$(experiment_filename_2).h5"))
chamber_roi_2 = read(orientation_correction_file,"chamber_roi")
global countour_array_2 = read(orientation_correction_file,"countour_array")
close(orientation_correction_file)


# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_3), "chamber_geometry_$(experiment_filename_3).h5"))
chamber_roi_3 = read(orientation_correction_file,"chamber_roi")
global countour_array_3 = read(orientation_correction_file,"countour_array")
close(orientation_correction_file)

In [ ]:
file_folder_1 = joinpath(data_path(ds_save_cy_1), "")
all_files = readdir(file_folder_1)
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = [occursin("A_dF", long_name_files[i])*occursin("neuron", long_name_files[i]) for i in 1:length(long_name_files)]
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(file_folder_1, save_file_name)
file = h5open(info_filename, "r")
place_cell_index_1 = HDF5.readmmap(file["place_cell_index"])
specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
close(file)


In [ ]:
save_file_name = "BVC_tel.h5"
bvc_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(bvc_filename, "r")
plot_candidate = copy(HDF5.readmmap(file["plot_candidate"])).+1
plot_candidate_map_1 = copy(HDF5.readmmap(file["plot_candidate_map_1"]))
plot_candidate_map_2 = copy(HDF5.readmmap(file["plot_candidate_map_2"]))
plot_candidate_map_3 = copy(HDF5.readmmap(file["plot_candidate_map_3"]))
plot_candidate_map_1_predict = copy(HDF5.readmmap(file["plot_candidate_map_1_predict"]))
plot_candidate_map_2_predict = copy(HDF5.readmmap(file["plot_candidate_map_2_predict"]))
plot_candidate_map_3_predict = copy(HDF5.readmmap(file["plot_candidate_map_3_predict"]))
plot_candidate_map_12_predict = copy(HDF5.readmmap(file["plot_candidate_map_12_predict"]))
corr_diff = copy(HDF5.readmmap(file["corr_diff"]))
close(file)

In [ ]:
fig_path = "/home/chuyu/fish_place_cell_paper/figures/output/sfigure2/bvc"
mkpath(fig_path)

In [ ]:
chosen_neruon_index = plot_candidate

In [ ]:
mask_valid = isfinite.(plot_candidate_map_1[:,:,1])
mask_invalid = isnan.(plot_candidate_map_1[:,:,1])
mask_valid_index = findall(mask_valid)
mask_valid_x = [x[1] for x in mask_valid_index]
mask_valid_y = [x[2] for x in mask_valid_index];
mask_valid_x_min = minimum(mask_valid_x)
mask_valid_x_max = maximum(mask_valid_x)
mask_valid_y_min = minimum(mask_valid_y)
mask_valid_y_max = maximum(mask_valid_y);

In [ ]:
chamber_roi_mask = Float32.(copy(chamber_roi_1))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
imshow(chamber_roi_mask)

In [ ]:
# example cells

valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

for (i, real_cell_index) in enumerate(chosen_neruon_index)
    fig = figure()
    map = plot_candidate_map_1[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,i]
    sp1 = imshow(map', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower", vmin=0, vmax=nanmaximum(map))
    # imshow(chamber_roi_mask', cmap="binary", origin="lower", vmin=0, vmax=3)
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    tight_layout()
    # colorbar()
    fig.savefig(joinpath(fig_path, "$(experiment_filename_1)_$real_cell_index map1.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

In [ ]:
# example cells
plot_candidate_map_1_predict[mask_invalid,:] .= NaN32
valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

for (i, real_cell_index) in enumerate(chosen_neruon_index)
    fig = figure()
    map = copy(plot_candidate_map_1_predict[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,i])
    sp1 = imshow(map', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower", vmin=0, vmax=nanmaximum(map))
    # imshow(chamber_roi_mask', cmap="binary", origin="lower", vmin=0, vmax=3)
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    tight_layout()
    fig.savefig(joinpath(fig_path, "$(experiment_filename_1)_$real_cell_index map1_predict.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

In [ ]:
mask_valid = isfinite.(plot_candidate_map_2[:,:,1])
mask_invalid = isnan.(plot_candidate_map_2[:,:,1])
mask_valid_index = findall(mask_valid)
mask_valid_x = [x[1] for x in mask_valid_index]
mask_valid_y = [x[2] for x in mask_valid_index];
mask_valid_x_min = minimum(mask_valid_x)
mask_valid_x_max = maximum(mask_valid_x)
mask_valid_y_min = minimum(mask_valid_y)
mask_valid_y_max = maximum(mask_valid_y);

In [ ]:
chamber_roi_mask = Float32.(copy(chamber_roi_2))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
imshow(chamber_roi_mask)

In [ ]:
# example cells

valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

for (i, real_cell_index) in enumerate(chosen_neruon_index)
    fig = figure()
    map = plot_candidate_map_2[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,i]
    sp1 = imshow(map', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower", vmin=0, vmax=nanmaximum(map))
    # imshow(chamber_roi_mask', cmap="binary", origin="lower", vmin=0, vmax=3)
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    tight_layout()
    # colorbar()
    fig.savefig(joinpath(fig_path, "$(experiment_filename_1)_$real_cell_index map2.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

In [ ]:
# example cells
plot_candidate_map_12_predict[mask_invalid,:] .= NaN32
valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

for (i, real_cell_index) in enumerate(chosen_neruon_index)
    fig = figure()
    map = copy(plot_candidate_map_12_predict[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,i])
    sp1 = imshow(map', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower", vmin=0, vmax=nanmaximum(map))
    # imshow(chamber_roi_mask', cmap="binary", origin="lower", vmin=0, vmax=3)
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    tight_layout()
    # colorbar()
    fig.savefig(joinpath(fig_path, "$(experiment_filename_1)_$real_cell_index map2_predict.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

In [ ]:
mask_valid = isfinite.(plot_candidate_map_3[:,:,1])
mask_invalid = isnan.(plot_candidate_map_3[:,:,1])
mask_valid_index = findall(mask_valid)
mask_valid_x = [x[1] for x in mask_valid_index]
mask_valid_y = [x[2] for x in mask_valid_index];
mask_valid_x_min = minimum(mask_valid_x)
mask_valid_x_max = maximum(mask_valid_x)
mask_valid_y_min = minimum(mask_valid_y)
mask_valid_y_max = maximum(mask_valid_y);

In [ ]:
chamber_roi_mask = Float32.(copy(chamber_roi_2))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
imshow(chamber_roi_mask)

In [ ]:
# example cells

valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

for (i, real_cell_index) in enumerate(chosen_neruon_index)
    fig = figure()
    map = plot_candidate_map_3[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,i]
    sp1 = imshow(map', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower", vmin=0, vmax=nanmaximum(map))
    # imshow(chamber_roi_mask', cmap="binary", origin="lower", vmin=0, vmax=3)
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    tight_layout()
    # colorbar()
    fig.savefig(joinpath(fig_path, "$(experiment_filename_1)_$real_cell_index map3.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

In [ ]:
fig_path = "/home/chuyu/fish_place_cell_paper/figures/output/sfigure2/non_bvc"
mkpath(fig_path)

In [ ]:
save_file_name = "BVC_tel.h5"
bvc_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(bvc_filename, "r")
plot_candidate = copy(HDF5.readmmap(file["plot_candidate_nonbvc"])) .+ 1
plot_candidate_map_1 = copy(HDF5.readmmap(file["plot_candidate_nonbvc_map_1"]))
plot_candidate_map_2 = copy(HDF5.readmmap(file["plot_candidate_nonbvc_map_2"]))
plot_candidate_map_3 = copy(HDF5.readmmap(file["plot_candidate_nonbvc_map_3"]))
plot_candidate_map_1_predict = copy(HDF5.readmmap(file["plot_candidate_nonbvc_map_1_predict"]))
plot_candidate_map_2_predict = copy(HDF5.readmmap(file["plot_candidate_nonbvc_map_2_predict"]))
plot_candidate_map_3_predict = copy(HDF5.readmmap(file["plot_candidate_nonbvc_map_3_predict"]))
plot_candidate_map_12_predict = copy(HDF5.readmmap(file["plot_candidate_nonbvc_map_12_predict"]))
corr_diff = copy(HDF5.readmmap(file["corr_diff"]))
close(file)

In [ ]:
chosen_neruon_index = plot_candidate

In [ ]:
specificity_population_z_1[plot_candidate]

In [ ]:
mask_valid = isfinite.(plot_candidate_map_1[:,:,1])
mask_invalid = isnan.(plot_candidate_map_1[:,:,1])
mask_valid_index = findall(mask_valid)
mask_valid_x = [x[1] for x in mask_valid_index]
mask_valid_y = [x[2] for x in mask_valid_index];
mask_valid_x_min = minimum(mask_valid_x)
mask_valid_x_max = maximum(mask_valid_x)
mask_valid_y_min = minimum(mask_valid_y)
mask_valid_y_max = maximum(mask_valid_y);

In [ ]:
chamber_roi_mask = Float32.(copy(chamber_roi_1))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
imshow(chamber_roi_mask)

In [ ]:
# example cells

valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

for (i, real_cell_index) in enumerate(chosen_neruon_index)
    fig = figure()
    map = plot_candidate_map_1[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,i]
    sp1 = imshow(map', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower", vmin=0, vmax=nanmaximum(map))
    # imshow(chamber_roi_mask', cmap="binary", origin="lower", vmin=0, vmax=3)
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    tight_layout()
    # colorbar()
    fig.savefig(joinpath(fig_path, "$(experiment_filename_1)_$real_cell_index map1.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

In [ ]:
# example cells
plot_candidate_map_1_predict[mask_invalid,:] .= NaN32
valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

for (i, real_cell_index) in enumerate(chosen_neruon_index)
    fig = figure()
    map = copy(plot_candidate_map_1_predict[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,i])
    sp1 = imshow(map', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower", vmin=0, vmax=nanmaximum(map))
    # imshow(chamber_roi_mask', cmap="binary", origin="lower", vmin=0, vmax=3)
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    tight_layout()
    fig.savefig(joinpath(fig_path, "$(experiment_filename_1)_$real_cell_index map1_predict.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

In [ ]:
mask_valid = isfinite.(plot_candidate_map_2[:,:,1])
mask_invalid = isnan.(plot_candidate_map_2[:,:,1])
mask_valid_index = findall(mask_valid)
mask_valid_x = [x[1] for x in mask_valid_index]
mask_valid_y = [x[2] for x in mask_valid_index];
mask_valid_x_min = minimum(mask_valid_x)
mask_valid_x_max = maximum(mask_valid_x)
mask_valid_y_min = minimum(mask_valid_y)
mask_valid_y_max = maximum(mask_valid_y);

In [ ]:
chamber_roi_mask = Float32.(copy(chamber_roi_2))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
imshow(chamber_roi_mask)

In [ ]:
# example cells

valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

for (i, real_cell_index) in enumerate(chosen_neruon_index)
    fig = figure()
    map = plot_candidate_map_2[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,i]
    sp1 = imshow(map', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower", vmin=0, vmax=nanmaximum(map))
    # imshow(chamber_roi_mask', cmap="binary", origin="lower", vmin=0, vmax=3)
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    tight_layout()
    # colorbar()
    fig.savefig(joinpath(fig_path, "$(experiment_filename_1)_$real_cell_index map2.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

In [ ]:
# example cells
plot_candidate_map_12_predict[mask_invalid,:] .= NaN32
valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

for (i, real_cell_index) in enumerate(chosen_neruon_index)
    fig = figure()
    map = copy(plot_candidate_map_12_predict[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,i])
    sp1 = imshow(map', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower", vmin=0, vmax=nanmaximum(map))
    # imshow(chamber_roi_mask', cmap="binary", origin="lower", vmin=0, vmax=3)
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    tight_layout()
    # colorbar()
    fig.savefig(joinpath(fig_path, "$(experiment_filename_1)_$real_cell_index map2_predict.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

In [ ]:
mask_valid = isfinite.(plot_candidate_map_3[:,:,1])
mask_invalid = isnan.(plot_candidate_map_3[:,:,1])
mask_valid_index = findall(mask_valid)
mask_valid_x = [x[1] for x in mask_valid_index]
mask_valid_y = [x[2] for x in mask_valid_index];
mask_valid_x_min = minimum(mask_valid_x)
mask_valid_x_max = maximum(mask_valid_x)
mask_valid_y_min = minimum(mask_valid_y)
mask_valid_y_max = maximum(mask_valid_y);

In [ ]:
chamber_roi_mask = Float32.(copy(chamber_roi_2))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
imshow(chamber_roi_mask)

In [ ]:
# example cells

valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

for (i, real_cell_index) in enumerate(chosen_neruon_index)
    fig = figure()
    map = plot_candidate_map_3[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,i]
    sp1 = imshow(map', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower", vmin=0, vmax=nanmaximum(map))
    # imshow(chamber_roi_mask', cmap="binary", origin="lower", vmin=0, vmax=3)
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    tight_layout()
    # colorbar()
    fig.savefig(joinpath(fig_path, "$(experiment_filename_1)_$real_cell_index map3.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end